~~~
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
~~~

# Using image embeddings to train a classification model

In this notebook we will train a model to classify findings from chest x-ray images in the [NIH-14 dataset](https://nihcc.app.box.com/v/ChestXray-NIHCC) based on embeddings generated by the [MedSigLIP model](https://developers.google.com/health-ai-developer-foundations/medsiglip).

NIH-14 contains chest x-ray images with labelled classes.

The MedSigLIP model is used to generate rich embeddings for medical images allowing us to train a machine learning model with less data and compute compared to training from scratch. Visit the [MedSigLIP page](https://developers.google.com/health-ai-developer-foundations/medsiglip) on the HAI-DEF site to learn more about the model.

We will frame this as a classification problem, where the model takes in a 6144 dimensional embedding and the label is X
<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/RSNA/AI-Deep-Learning-Lab-2025/blob/main/DL%20Lab%20-%20Leveraging%20pre-trained%20embeddings%20models%20for%20your%20use%20cases%20and%20your%20data/RSNA_Embeddings_Workshop.ipynb">
      <img alt="Google Colab logo" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="32px"><br> Run in Google Colab
    </a>
  </td>
</tr></tbody></table>

## Setup

To complete this tutorial, you'll need to have a runtime with [sufficient resources](https://ai.google.dev/gemma/docs/core#sizes) to run the MedGemma model.

You can try out MedGemma 4B for free in Google Colab using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

**Note**: To run the demo with MedGemma 27B in Google Colab, you will need a runtime with an A100 GPU and use 4-bit quantization to reduce memory usage. The performance of quantized versions has not been evaluated.

In [ ]:
!pip install -U bitsandbytes accelerate

In [ ]:
import collections
import io
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import storage
import json
from google.colab import output


In [ ]:
import pandas as pd
import json
from google.cloud import storage

# Configuration
gcs_path = "gs://healthai-us/medsiglip/cxr-14-embeddings-with-labels.jsonl"
local_filename = "cxr-14-embeddings-with-labels.jsonl"

# Parse GCS Path
path_parts = gcs_path.replace("gs://", "").split("/", 1)
bucket_name = path_parts[0]
blob_name = path_parts[1]

print(f"Downloading from Bucket: {bucket_name}, Blob: {blob_name}")

# Initialize Client (Anonymous for public access)
client = storage.Client.create_anonymous_client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

# Download to local file
blob.download_to_filename(local_filename)
print(f"Downloaded to {local_filename}")

# Read the file
data = []
errors = 0

with open(local_filename, 'r') as f:
    for i, line in enumerate(f):
        if not line.strip():
            continue
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            errors += 1
            if errors <= 5:
                print(f"Skipping error on line {i+1}: {e}")

if errors > 5:
    print(f"... and {errors - 5} more errors.")

df_embeddings = pd.DataFrame(data)
print(f"Successfully loaded {len(df_embeddings)} rows (skipped {errors} invalid lines).")
display(df_embeddings.head())

Analyze the distribution of labels

In [ ]:
# Define metadata columns to exclude
meta_cols = ['path', 'embedding', 'Patient ID', 'Patient Age', 'Patient Sex', 'Follow-up #', 'subject_id']

# Identify label columns (all columns that are not metadata)
label_cols = [c for c in df_embeddings.columns if c not in meta_cols]

print(f"Calculating counts for {len(label_cols)} labels...")

# Compute counts
counts_data = []
for col in label_cols:
    # values are 0.0 or 1.0
    positives = int(df_embeddings[col].sum())
    negatives = len(df_embeddings) - positives
    counts_data.append({
        'Finding': col,
        'Positives': positives,
        'Negatives': negatives
    })

# Create DataFrame and sort
counts_df_final = pd.DataFrame(counts_data)
counts_df_final = counts_df_final.sort_values(by='Positives', ascending=False).reset_index(drop=True)

display(counts_df_final)

## Prepare Data

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# 1. Condition Dropdown
conditions = counts_df_final['Finding'].tolist()

initial_dropdown_value = conditions[5] # Default to the first condition

# Check if 'selected_condition' variable exists in the global scope
# and if its current value is a valid option in the 'conditions' list.
if 'selected_condition' in globals() and globals()['selected_condition'] in conditions:
    initial_dropdown_value = globals()['selected_condition']

condition_dropdown = widgets.Dropdown(
    options=conditions,
    value=initial_dropdown_value,
    description='Condition:',
    disabled=False,
)

# Output widget for displaying dynamic stats
stats_output = widgets.Output()

def on_condition_change(change):
    global selected_condition # Ensure this modifies the global selected_condition
    selected_condition = change['new']
    n_pos = int(df_embeddings[selected_condition].sum())
    n_total = len(df_embeddings)

    with stats_output:
        stats_output.clear_output()
        print(f"  → Found {n_pos} positive examples out of {n_total} total.")
        if n_pos < 50:
            print(f"  ★ WARNING: Low positive count.")

# Bind the event listener
condition_dropdown.observe(on_condition_change, names='value')

# 2. Negative Ratio Input
initial_neg_ratio = 3.0
if 'neg_ratio_input' in globals():
    initial_neg_ratio = globals()['neg_ratio_input'].value

neg_ratio_input = widgets.FloatText(
    value=initial_neg_ratio,
    description='Neg Ratio:',
    step=0.1
)

# 3. Train Split Input
initial_train_split = 0.7
if 'train_split_input' in globals():
    initial_train_split = globals()['train_split_input'].value

train_split_input = widgets.FloatText(
    value=initial_train_split,
    description='Train Split:',
    step=0.05
)

# Manually trigger the initial update to display stats for the dropdown's initial value
# and to ensure `selected_condition` is set for subsequent cells.
on_condition_change({'new': initial_dropdown_value})

print("Select your parameters:")
display(condition_dropdown, stats_output, neg_ratio_input, train_split_input)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# =================================================================================
# REPLACEMENT CELL: Split by Patient ID -> Balance Train Only -> Keep Val Natural
# =================================================================================

# 1. Get Inputs
selected_condition = condition_dropdown.value
neg_ratio = neg_ratio_input.value
train_split_ratio = train_split_input.value

print(f"Condition: {selected_condition}")
print(f"Balancing Training Data to 1:{neg_ratio} ratio.")
print(f"Keeping Validation Data with natural imbalance (using all available images).")

# 2. Split Patient IDs (Stratified)
# We split the PATIENTS first, so no patient exists in both sets.
patient_ids = df_embeddings['Patient ID'].unique()
# Determine if a patient is "positive" if they have at least one positive image
patient_labels = df_embeddings.groupby('Patient ID')[selected_condition].max()

train_ids, val_ids = train_test_split(
    patient_ids,
    train_size=train_split_ratio,
    random_state=42,
    stratify=patient_labels
)

# 3. Create Initial DataFrames (All images for the specific patients)
train_df_raw = df_embeddings[df_embeddings['Patient ID'].isin(train_ids)]
val_df = df_embeddings[df_embeddings['Patient ID'].isin(val_ids)].copy()

# ---------------------------------------------------------
# 4. Balance ONLY the Training Set
# ---------------------------------------------------------
# Isolate Positives and Negatives in Training
train_pos = train_df_raw[train_df_raw[selected_condition] == 1]
train_neg = train_df_raw[train_df_raw[selected_condition] == 0]

# Calculate how many negatives we want for training
n_pos_train = len(train_pos)
n_neg_target = int(n_pos_train * neg_ratio)
n_neg_available = len(train_neg)

# Sample negatives (or take all if we don't have enough)
if n_neg_target > n_neg_available:
    print(f"Note: Training set requested {n_neg_target} negatives, but only {n_neg_available} available.")
    n_neg_final = n_neg_available
else:
    n_neg_final = n_neg_target

train_neg_sampled = train_neg.sample(n=n_neg_final, random_state=42)

# Combine to create final Training Set
train_df = pd.concat([train_pos, train_neg_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# ---------------------------------------------------------
# 5. Prepare Output Arrays
# ---------------------------------------------------------
X_train = np.stack(train_df['embedding'].values)
y_train = train_df[selected_condition].values

X_val = np.stack(val_df['embedding'].values)
y_val = val_df[selected_condition].values

# ---------------------------------------------------------
# 6. Stats & Verification
# ---------------------------------------------------------
def print_set_stats(name, df, condition):
    n_p = df[condition].sum()
    n_n = len(df) - n_p
    ratio = n_n / n_p if n_p > 0 else 0
    print(f"{name}: {len(df)} images ({int(n_p)} Pos, {int(n_n)} Neg) [Ratio 1:{ratio:.1f}]")

print(f"\nFinal Dataset Stats:")
print_set_stats("Train Set (Balanced)", train_df, selected_condition)
print_set_stats("Tune Set (Natural) ", val_df, selected_condition)

# Leakage Check
overlap = set(train_df['Patient ID']).intersection(set(val_df['Patient ID']))
if len(overlap) == 0:
    print("\n✓ SUCCESS: No patient leakage detected.")
else:
    print(f"\n☠ FAIL: Leakage detected ({len(overlap)} patients).")

## Train Logistic Regression

### Subtask:
Train a logistic regression model on the prepared training data.


**Reasoning**:
Train a logistic regression model using the prepared training data and verify the class labels.



In [ ]:
from sklearn.linear_model import LogisticRegression

# 1. Instantiate the model
log_reg_model = LogisticRegression(random_state=42, max_iter=1000)

# 2. Fit the model
log_reg_model.fit(X_train, y_train)

# 3. Verify classes
print(f"Model classes: {log_reg_model.classes_}")

## Evaluate Logistic Regression

### Subtask:
Evaluate the trained Logistic Regression model on the validation set using metrics like ROC AUC, Accuracy, and Confusion Matrix.


**Reasoning**:
Evaluate the trained model on the validation data using various metrics and visualization.



In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score

# 1. Generate predictions (Keep your existing code)
y_pred_proba = log_reg_model.predict_proba(X_val)[:, 1]
y_pred_binary = log_reg_model.predict(X_val)

# 2. Calculate metrics
roc_auc = roc_auc_score(y_val, y_pred_proba)
accuracy = accuracy_score(y_val, y_pred_binary)

print(f"ROC AUC Score: {roc_auc:.4f}")
print(f"Accuracy Score: {accuracy:.4f}")

# ---------------------------------------------------------
# 3. Create Interactive Plot using Plotly
# ---------------------------------------------------------

# Calculate the coordinates for the curve
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

# Create the figure
fig = go.Figure()

# Add the ROC Curve line
fig.add_trace(go.Scatter(
    x=fpr,
    y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.4f})',
    # This 'text' argument allows us to display the threshold on hover
    text=thresholds,
    # This formats the hover tooltip
    hovertemplate=(
        '<b>Threshold:</b> %{text:.4f}<br>' +
        '<b>Sensitivity (TPR):</b> %{y:.4f}<br>' +
        '<b>1-Specificity (FPR):</b> %{x:.4f}<extra></extra>'
    )
))

# Add the diagonal "No Skill" line
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Chance'
))

# Update layout for labels and sizing
fig.update_layout(
    title=f'ROC Curve for {selected_condition}',
    xaxis_title='False Positive Rate (1 - Specificity)',
    yaxis_title='True Positive Rate (Sensitivity)',
    width=700,
    height=600,
    showlegend=True,
    template="plotly_white" # Clean white background
)

fig.show()

## Train Neural Network

### Subtask:
Define and train a TensorFlow/Keras binary classification model using the training data.


**Reasoning**:
Define and train a TensorFlow/Keras binary classification model using the training data as requested.



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers
import matplotlib.pyplot as plt


# 1. Create TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(128)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(128)

# 2. Define the model
weight_decay = 0#1e-5

inputs = tf.keras.Input(shape=(1152,))
hidden = layers.Dense(512,
                      kernel_regularizer=regularizers.l2(weight_decay),
                      bias_regularizer=regularizers.l2(weight_decay),
                      activation="relu")(inputs)
hidden = layers.Dropout(0.05)(hidden)
hidden = layers.Dense(256,
                      kernel_regularizer=regularizers.l2(weight_decay),
                      bias_regularizer=regularizers.l2(weight_decay),
                      activation="relu")(hidden)
hidden = layers.Dropout(0.1)(hidden)
outputs = layers.Dense(1, activation="sigmoid")(hidden)

model = tf.keras.Model(inputs, outputs)

# 3. Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name='auc')]
)

# 4. Train the model
print(model.summary())
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30
)

# Access the history dictionary
history_dict = history.history

# Extract loss values
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(epochs, loss_values, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss_values, 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## Evaluate Neural Network


### Subtask:
Evaluate the trained Neural Network on the validation set using ROC AUC, Confusion Matrix, and Loss curves.


In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score

# 1. Generate predictions using the Neural Network
# model.predict returns a (N, 1) array, so we flatten it
y_pred_proba = model.predict(X_val, verbose=0).ravel()
y_pred_binary = (y_pred_proba > 0.5).astype(int)

# 2. Calculate metrics
roc_auc = roc_auc_score(y_val, y_pred_proba)
accuracy = accuracy_score(y_val, y_pred_binary)

print(f"ROC AUC Score: {roc_auc:.4f}")
print(f"Accuracy Score: {accuracy:.4f}")

# ---------------------------------------------------------
# 3. Create Interactive Plot using Plotly
# ---------------------------------------------------------

# Calculate the coordinates for the curve
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

# Create the figure
fig = go.Figure()

# Add the ROC Curve line
fig.add_trace(go.Scatter(
    x=fpr,
    y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.4f})',
    # This 'text' argument allows us to display the threshold on hover
    text=thresholds,
    # This formats the hover tooltip
    hovertemplate=(
        '<b>Threshold:</b> %{text:.4f}<br>' +
        '<b>Sensitivity (TPR):</b> %{y:.4f}<br>' +
        '<b>1-Specificity (FPR):</b> %{x:.4f}<extra></extra>'
    )
))

# Add the diagonal "No Skill" line
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Chance'
))

# Update layout for labels and sizing
fig.update_layout(
    title=f'ROC Curve for {selected_condition} (Neural Network)',
    xaxis_title='False Positive Rate (1 - Specificity)',
    yaxis_title='True Positive Rate (Sensitivity)',
    width=700,
    height=600,
    showlegend=True,
    template="plotly_white" # Clean white background
)

import pandas as pd
import numpy as np

# 1. Generate Predictions
# We run the trained Neural Network (model) on the validation features (X_val)
print("Generating predictions on Validation Set...")
y_pred_proba = model.predict(X_val, verbose=0).ravel()

# 2. Create the Metadata DataFrame
# We clone val_df so we don't mess up the original data
df_meta = val_df.copy().reset_index(drop=True)

# 3. Add Key Columns
# 'Actual': The Ground Truth (0 or 1)
df_meta['Actual'] = df_meta[selected_condition].values

# 'Probability': The Neural Net's output score (0.0 to 1.0)
df_meta['Probability'] = y_pred_proba

# 4. Construct Full Image Paths
# The 'path' column in the JSONL file is just the filename (e.g., "0000001_000.png")
# We need the full Google Cloud Storage path for the image loader to work.
base_gcs_url = "gs://healthai-us/nih-cxr-14/png/"

# Ensure we handle cases where the path might already have the prefix (rare, but safe)
def format_path(p):
    if p.startswith("gs://"): return p
    return base_gcs_url + p

df_meta['gcs_path'] = df_meta['path'].apply(format_path)

# 5. Create a UI-friendly ID
# We use the DataFrame index as a simple numeric ID for the slider/dropdown
df_meta['ID'] = df_meta.index

# 6. Filter to clean up memory
# We only keep what the Dashboard needs
cols_to_keep = ['ID', 'Patient Age', 'Patient Sex', 'Actual', 'Probability', 'gcs_path']
df_meta = df_meta[cols_to_keep]

fig.show()

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, confusion_matrix

# ==========================================
# 1. WIDGET DEFINITIONS
# ==========================================

# A. Header
header_html = widgets.HTML(
    value=f"<h2 style='margin:0; color:#333; font-family:sans-serif;'>Analysis Dashboard: <span style='color:#007bff'>{selected_condition}</span></h2>"
)

# B. Threshold Controls (Linked Trio)
btn_minus = widgets.Button(description='-', layout=widgets.Layout(width='35px'))
btn_plus = widgets.Button(description='+', layout=widgets.Layout(width='35px'))

txt_thresh = widgets.BoundedFloatText(
    value=0.50, min=0.0, max=1.0, step=0.01,
    layout=widgets.Layout(width='60px'),
    continuous_update=False
)

slider_thresh = widgets.FloatSlider(
    value=0.50, min=0.0, max=1.0, step=0.01,
    readout=False,
    layout=widgets.Layout(width='200px'),
    continuous_update=True
)

widgets.jslink((txt_thresh, 'value'), (slider_thresh, 'value'))

# C. Demographics
age_slider = widgets.IntRangeSlider(
    value=[df_meta['Patient Age'].min(), df_meta['Patient Age'].max()],
    min=df_meta['Patient Age'].min(),
    max=df_meta['Patient Age'].max(),
    step=1,
    description='Age:',
    continuous_update=False,
    layout=widgets.Layout(width='300px')
)

sex_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df_meta['Patient Sex'].unique().tolist()),
    value='All',
    description='Sex:',
    layout=widgets.Layout(width='120px')
)

# D. Outputs
metrics_display = widgets.HTML(value="Initialize...")
id_input = widgets.BoundedIntText(
    value=df_meta['ID'].min(), min=0, max=df_meta['ID'].max(),
    description='ID:', layout=widgets.Layout(width='150px')
)

out_plot = widgets.Output(layout=widgets.Layout(width='45%', height='600px', display='flex', align_items='center', justify_content='center'))
out_table = widgets.Output(layout=widgets.Layout(width='55%', height='600px', overflow='scroll', border='1px solid #eee'))
out_image = widgets.Output(layout=widgets.Layout(width='100%', min_height='500px', border='1px solid #ddd', margin='10px 0'))

# ==========================================
# 2. LOGIC FUNCTIONS
# ==========================================
from google.cloud import storage

def get_image_from_gcs(gcs_path, project_id=None):
    try:
        path_parts = gcs_path.replace('gs://', '').split('/', 1)
        # Create an anonymous client if no project_id is needed/provided
        if project_id:
            client = storage.Client(project=project_id)
        else:
            client = storage.Client.create_anonymous_client()

        bucket = client.bucket(path_parts[0])
        blob = bucket.blob(path_parts[1])
        return Image.open(io.BytesIO(blob.download_as_bytes()))
    except Exception as e:
        print(f"GCS Load Error: {e}")
        return None

def adjust_threshold(change_amt):
    new_val = round(txt_thresh.value + change_amt, 2)
    if 0.0 <= new_val <= 1.0:
        txt_thresh.value = new_val

btn_minus.on_click(lambda b: adjust_threshold(-0.01))
btn_plus.on_click(lambda b: adjust_threshold(0.01))

def get_filtered_data():
    min_age, max_age = age_slider.value
    sex = sex_dropdown.value
    mask = (df_meta['Patient Age'] >= min_age) & (df_meta['Patient Age'] <= max_age)
    if sex != 'All':
        mask = mask & (df_meta['Patient Sex'] == sex)
    return df_meta[mask].copy().sort_values('Probability', ascending=False)

def update_dashboard(change=None):
    t = txt_thresh.value
    df_subset = get_filtered_data()

    if len(df_subset) == 0:
        metrics_display.value = "<b style='color:red'>No data matches filters.</b>"
        with out_plot: clear_output(); print("No Data")
        with out_table: clear_output(); print("No Data")
        return

    # Metrics
    preds = (df_subset['Probability'] >= t).astype(int)
    try:
        tn, fp, fn, tp = confusion_matrix(df_subset['Actual'], preds, labels=[0,1]).ravel()
        sens = tp/(tp+fn) if (tp+fn)>0 else 0
        spec = tn/(tn+fp) if (tn+fp)>0 else 0
    except:
        sens, spec = 0, 0

    metrics_display.value = f"""
    <div style="font-family: sans-serif; padding: 12px; background: #f8f9fa; border: 1px solid #e9ecef; border-radius: 5px; display: flex; gap: 30px; align-items: center; margin-bottom: 10px;">
        <span style="font-size:14px"><b>Count:</b> {len(df_subset)}</span>
        <span style="font-size:14px"><b>Threshold:</b> {t:.2f}</span>
        <span style="font-size:14px; color: #28a745;"><b>Sensitivity:</b> {sens:.3f}</span>
        <span style="font-size:14px; color: #007bff;"><b>Specificity:</b> {spec:.3f}</span>
    </div>
    """

    # Update Plot
    fpr, tpr, _ = roc_curve(df_subset['Actual'], df_subset['Probability']) if len(df_subset['Actual'].unique()) > 1 else ([0,1], [0,1], 0)
    auc_score = np.trapezoid(tpr, fpr)

    with out_plot:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(9, 7))
        ax.plot(fpr, tpr, color='#007bff', lw=3, label=f'AUC={auc_score:.2f}')
        ax.plot([0, 1], [0, 1], color='gray', linestyle='--')
        ax.plot(1-spec, sens, marker='o', color='red', markersize=14, markeredgecolor='black', zorder=10)
        ax.set_title(f'ROC Curve: {selected_condition}', fontsize=14, fontweight='bold')
        ax.set_xlabel('1 - Specificity')
        ax.set_ylabel('Sensitivity')
        ax.legend(loc="lower right")
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.show()

    # --- IMPROVED TABLE LOGIC ---
    with out_table:
        clear_output(wait=True)

        # Split into Above and Below threshold
        df_above = df_subset[df_subset['Probability'] >= t]
        df_below = df_subset[df_subset['Probability'] < t]

        # Smart Slice: Top 20 + [50 above ... 50 below] + Bottom 20
        # This ensures we see the threshold context AND the extremes

        slice_top = df_subset.head(20)
        slice_above_thresh = df_above.tail(100)
        slice_below_thresh = df_below.head(100)
        slice_bottom = df_subset.tail(20)

        # Concat and Drop Duplicates (in case list is small and slices overlap)
        df_display = pd.concat([slice_top, slice_above_thresh, slice_below_thresh, slice_bottom])
        df_display = df_display.drop_duplicates().sort_values('Probability', ascending=False)

        html_rows = ""
        line_drawn = False

        for _, row in df_display.iterrows():
            # DRAW THRESHOLD LINE
            if not line_drawn and row['Probability'] < t:
                html_rows += f'<tr id="thresh-line" style="background:#ffcccc; border-top:3px solid red; border-bottom:3px solid red; font-weight:bold; color:#cc0000;"><td colspan="7" style="padding:8px;">⬇ CUTOFF THRESHOLD ({t:.2f}) ⬇</td></tr>'
                line_drawn = True

            pred = 1 if row['Probability'] >= t else 0
            if row['Actual'] == 1 and pred == 1: cls, lbl = "tp", "TP"
            elif row['Actual'] == 0 and pred == 0: cls, lbl = "tn", "TN"
            elif row['Actual'] == 0 and pred == 1: cls, lbl = "fp", "FP"
            elif row['Actual'] == 1 and pred == 0: cls, lbl = "fn", "FN"
            else: cls, lbl = "", "?"

            js_click = f"google.colab.kernel.invokeFunction('select_case', [{row['ID']}], {{}})"

            bg_color = {"tp": "#e6ffe6", "tn": "#ffffff", "fp": "#ffe6e6", "fn": "#fff5e6"}.get(cls, "#fff")
            font_color = {"tp": "green", "tn": "navy", "fp": "#d9534f", "fn": "#f0ad4e"}.get(cls, "black")
            fw = "bold" if cls in ["fp", "fn"] else "normal"

            html_rows += f'''
            <tr style="background-color:{bg_color}; color:{font_color}; font-weight:{fw}; cursor:pointer; border-bottom:1px solid #f0f0f0;" onclick="{js_click}" onmouseover="this.style.backgroundColor='#ffffcc'" onmouseout="this.style.backgroundColor='{bg_color}'">
                <td style="padding:6px;">{row['ID']}</td>
                <td style="padding:6px;">{row['Probability']:.3f}</td>
                <td style="padding:6px;">{int(row['Actual'])}</td>
                <td style="padding:6px;">{pred}</td>
                <td style="padding:6px;">{lbl}</td>
                <td style="padding:6px;">{row['Patient Age']}</td>
                <td style="padding:6px;">{row['Patient Sex']}</td>
            </tr>'''

        table_html = f"""
        <style> th {{ position: sticky; top: 0; background: white; z-index: 10; border-bottom: 2px solid #333; }} </style>
        <table style="width:100%; border-collapse:collapse; font-family: 'Segoe UI', sans-serif; font-size:13px; text-align:center;">
            <thead>
                <tr style="background:#f8f9fa;">
                    <th style="padding:8px;">ID</th>
                    <th style="padding:8px;">Model Score</th>
                    <th style="padding:8px;">True</th>
                    <th style="padding:8px;">Pred</th>
                    <th style="padding:8px;">Type</th>
                    <th style="padding:8px;">Age</th>
                    <th style="padding:8px;">Sex</th>
                </tr>
            </thead>
            <tbody>{html_rows}</tbody>
        </table>
        """
        display(HTML(table_html))

        # SCROLL TO LINE
        if line_drawn:
            display(Javascript("""
            setTimeout(function() {
                var el = document.getElementById("thresh-line");
                if(el) { el.scrollIntoView({behavior: "auto", block: "center", inline: "nearest"}); }
            }, 300);
            """))

# Load Image Logic
def load_case_image(id_val):
    idx = int(id_val)
    id_input.value = idx
    row_subset = df_meta[df_meta['ID'] == idx]

    with out_image:
        clear_output(wait=True)
        if len(row_subset) == 0: print("ID not found."); return
        row = row_subset.iloc[0]
        try:
            img = get_image_from_gcs(row['gcs_path'], project_id=None)
            if img:
                plt.figure(figsize=(7, 7))
                plt.imshow(img, cmap='gray')
                plt.axis('off')
                curr_pred = 1 if row['Probability'] >= txt_thresh.value else 0
                title = f"ID: {idx} ({row['Patient Sex']}, {row['Patient Age']}y)\nActual: {row['Actual']} | Pred: {curr_pred} (Score={row['Probability']:.3f})"
                plt.title(title, fontsize=16, color='#28a745' if row['Actual']==curr_pred else '#dc3545')
                plt.show()
            else:
                print("Image load returned None.")
        except Exception as e:
            print(f"Error loading image: {e}")

output.register_callback('select_case', load_case_image)

# ==========================================
# 3. LAYOUT ASSEMBLY
# ==========================================

txt_thresh.observe(update_dashboard, names='value')
age_slider.observe(update_dashboard, names='value')
sex_dropdown.observe(update_dashboard, names='value')
id_input.observe(lambda c: load_case_image(c['new']), names='value')

controls_box = widgets.HBox([
    widgets.VBox([
        widgets.Label("Threshold:", style={'font_weight': 'bold'}),
        widgets.HBox([btn_minus, txt_thresh, btn_plus, slider_thresh])
    ]),
    widgets.HTML("&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;"),
    widgets.VBox([
        widgets.Label("Demographics:", style={'font_weight': 'bold'}),
        widgets.HBox([age_slider, sex_dropdown])
    ])
], layout=widgets.Layout(align_items='center', margin='10px 0'))

header_box = widgets.HBox([header_html], layout=widgets.Layout(border_bottom='2px solid #ddd', padding='10px 0'))

ui_final = widgets.VBox([
    header_box,
    controls_box,
    metrics_display,
    widgets.HBox([out_plot, out_table], layout=widgets.Layout(width='100%', justify_content='space-between')),
    widgets.HTML("<hr><h4 style='color:#555;'>Selected Case Viewer</h4>"),
    widgets.HBox([widgets.Label("Selected ID:"), id_input]),
    out_image
])

update_dashboard()
display(ui_final)

## Zero-shot Text Embeddings

## Get access to MedSigLIP

Before you get started, make sure that you have access to MedGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [MedSiglip model page](https://huggingface.co/google/medsiglip-448) and accept the usage conditions.

### Authenticate with Hugging Face

Generate a Hugging Face `read` access token by going to [settings](https://huggingface.co/settings/tokens).

If you are using Google Colab, add your access token to the Colab Secrets manager to securely store it. If not, proceed to run the cell below to authenticate with Hugging Face.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.

In [ ]:
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from transformers import AutoProcessor, AutoModel
from sklearn.metrics import roc_curve, roc_auc_score

# ==========================================
# 1. SETUP MODEL
# ==========================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading MedSigLIP text encoder on {device}...")

processor = AutoProcessor.from_pretrained("google/medsiglip-448")
model = AutoModel.from_pretrained("google/medsiglip-448").to(device)

# ==========================================
# 2. MULTI-PROMPT DICTIONARY
# ==========================================
# We use multiple variations to create a robust "Ensemble" embedding
prompt_ensemble = {
    'Pneumothorax': {
        'pos': ['pneumothorax', 'large pneumothorax', 'small pneumothorax', 'collapsed lung', 'air in the pleural space', 'visible visceral pleural edge'],
        'neg': ['no pneumothorax', 'normal lung', 'clear lungs', 'no evidence of pneumothorax', 'lung fields are clear']
    },
    'Effusion': {
        'pos': ['pleural effusion', 'fluid in the pleural space', 'blunting of the costophrenic angle', 'meniscus sign', 'large pleural effusion'],
        'neg': ['no pleural effusion', 'costophrenic angles are sharp', 'normal chest x-ray', 'no fluid overload']
    },
    'Infiltration': {
        'pos': ['infiltration', 'pulmonary infiltrate', 'airspace opacity', 'patchy opacities', 'consolidation or infiltration'],
        'neg': ['no infiltration', 'lungs are clear', 'no airspace opacity', 'normal lung parenchyma']
    },
    'Edema': {
        'pos': ['pulmonary edema', 'interstitial edema', 'vascular congestion', 'curly b lines', 'fluid overload', 'prominent pulmonary vasculature'],
        'neg': ['no pulmonary edema', 'no vascular congestion', 'normal pulmonary vasculature', 'dry lungs']
    },
    'Atelectasis': {
        'pos': ['atelectasis', 'lung collapse', 'loss of volume', 'linear opacity', 'plate-like atelectasis'],
        'neg': ['no atelectasis', 'fully expanded lungs', 'normal lung volumes']
    },
    'Nodule': {
        'pos': ['pulmonary nodule', 'lung nodule', 'solitary pulmonary nodule', 'round opacity', 'mass or nodule'],
        'neg': ['no nodule', 'no masses', 'clear lungs', 'normal chest']
    },
    'Mass': {
        'pos': ['lung mass', 'large mass', 'tumor', 'malignancy', 'large opacity'],
        'neg': ['no mass', 'no tumor', 'benign chest x-ray']
    }
}

# ==========================================
# 3. HELPER: AVERAGE TEXT EMBEDDING
# ==========================================

def get_averaged_text_embedding(text_list):
    """
    Takes a list of strings, generates embeddings for all of them,
    averages them, and returns a single normalized vector (1152,).
    """
    # Tokenize all prompts at once
    inputs = processor(text=text_list, images=None, return_tensors="pt", padding="max_length", truncation=True).to(device)

    with torch.no_grad():
        # Get raw features (Shape: [Batch_Size, 1152])
        outputs = model.get_text_features(**inputs)

        # 1. Normalize individual vectors first (Standard CLIP practice)
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)

        # 2. Average them into one vector (Mean Pooling)
        # Shape: [1, 1152]
        avg_output = torch.mean(outputs, dim=0)

        # 3. Normalize the final averaged vector
        avg_output = avg_output / avg_output.norm(p=2, dim=-1, keepdim=True)

    return avg_output.cpu().numpy()

def compute_zero_shot_score(image_emb, pos_emb, neg_emb):
    # Normalize Image
    img_norm = image_emb / (np.linalg.norm(image_emb) + 1e-8)
    # Cosine Sim
    pos_sim = np.dot(img_norm, pos_emb)
    neg_sim = np.dot(img_norm, neg_emb)
    return pos_sim - neg_sim

# ==========================================
# 4. EXECUTION
# ==========================================

target_condition = selected_condition
print(f"\n--- Running Ensemble Zero-Shot for: {target_condition} ---")

# Get prompts (Fallback to generic if not in dictionary)
if target_condition in prompt_ensemble:
    pos_list = prompt_ensemble[target_condition]['pos']
    neg_list = prompt_ensemble[target_condition]['neg']
    print(f"✅ Using Ensemble Prompts ({len(pos_list)} Pos, {len(neg_list)} Neg)")
else:
    print(f"⚠️ Condition not in ensemble map. Using generic prompts.")
    pos_list = [f"{target_condition}", f"finding of {target_condition}"]
    neg_list = [f"no {target_condition}", f"no evidence of {target_condition}"]

# Generate Ensembled Embeddings
pos_text_emb = get_averaged_text_embedding(pos_list)
neg_text_emb = get_averaged_text_embedding(neg_list)

# Compute Scores
zs_scores = []
print(f"Computing scores for {len(val_df)} validation images...")

for idx, row in val_df.iterrows():
    img_emb = np.array(row['embedding'])
    score = compute_zero_shot_score(img_emb, pos_text_emb, neg_text_emb)
    zs_scores.append(score)

# ==========================================
# 5. PLOTTING
# ==========================================

y_true = val_df[target_condition].values
y_scores = np.array(zs_scores)
roc_auc = roc_auc_score(y_true, y_scores)
print(f"\n>> Ensemble Zero-Shot AUC: {roc_auc:.4f}")

fpr, tpr, thresholds = roc_curve(y_true, y_scores)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=fpr, y=tpr, mode='lines',
    name=f'Ensemble ZS (AUC={roc_auc:.4f})',
    line=dict(width=3, color='#8e44ad'), # Purple for Ensemble
    text=thresholds,
    hovertemplate='<b>Thresh:</b> %{text:.4f}<br><b>Sens:</b> %{y:.4f}<br><b>1-Spec:</b> %{x:.4f}<extra></extra>'
))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(color='black', dash='dash'), name='Chance'))

fig.update_layout(
    title=f'Ensemble Zero-Shot ROC: {target_condition}',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    width=700, height=600, template="plotly_white",
    legend=dict(yanchor="bottom", y=0.02, xanchor="right", x=0.98, bgcolor="rgba(255,255,255,0.9)", bordercolor="lightgrey")
)
fig.show()

# Explore with MedGemma

### Get access to MedGemma

Before you get started, make sure that you have access to MedGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [MedGemma model page](https://huggingface.co/google/medgemma-4b-it) and accept the usage conditions.

In [ ]:

from transformers import pipeline, BitsAndBytesConfig
import torch

# ==========================================
# 1. CONFIGURE & LOAD MEDGEMMA
# ==========================================
model_id = "google/medgemma-4b-it"
use_quantization = False  # @param {type: "boolean"}
model_kwargs = dict(
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

if use_quantization:
  print(f"Loading {model_id} with 4-bit quantization...")
  bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)
  model_kwargs["quantization_config"] = bnb_config

  # We use the pipeline API for easiest integration
pipe = pipeline(
    "image-text-to-text",
    model=model_id,
    model_kwargs=model_kwargs
)

print("MedGemma Loaded Successfully!")

# Wrapper function for cleaner usage
def run_medgemma(pil_image, prompt_text):
    messages = [
        {"role": "user", "content": [
            {"type": "text", "text": prompt_text},
            {"type": "image", "image": pil_image}
        ]}
    ]
    # Run generation
    output = pipe(text=messages, max_new_tokens=300)
    return output[0]["generated_text"][-1]["content"]

Explore how MedGemma matches the MedSigLIP model

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript, Markdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 1. WIDGET DEFINITIONS
# ==========================================

# A. Header
header_html = widgets.HTML(
    value=f"<h2 style='margin:0; color:#2c3e50; font-family:sans-serif;'>AI Radiologist Assistant: <span style='color:#e67e22'>MedGemma 4B</span></h2>"
)

# B. Demographics
age_slider = widgets.IntRangeSlider(
    value=[df_meta['Patient Age'].min(), df_meta['Patient Age'].max()],
    min=df_meta['Patient Age'].min(),
    max=df_meta['Patient Age'].max(),
    step=1,
    description='Age:',
    continuous_update=False,
    layout=widgets.Layout(width='350px')
)

sex_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df_meta['Patient Sex'].unique().tolist()),
    value='All',
    description='Sex:',
    layout=widgets.Layout(width='150px')
)

# C. MedGemma Controls
default_prompt_text = f"Describe this chest X-ray. Is there evidence of {selected_condition}? Explain your reasoning. Start by answering yes or no."

txt_prompt = widgets.Textarea(
    value=default_prompt_text,
    placeholder='Enter instructions for the AI...',
    layout=widgets.Layout(width='98%', height='100px')
)

btn_generate = widgets.Button(
    description="Generate Report",
    button_style='success',
    icon='magic',
    layout=widgets.Layout(width='98%', height='40px')
)

# D. Outputs
out_table = widgets.Output(layout=widgets.Layout(width='100%', height='350px', overflow='scroll', border='1px solid #eee', margin='0 0 20px 0'))
id_input = widgets.BoundedIntText(value=df_meta['ID'].min(), min=0, max=df_meta['ID'].max(), description='ID:')
out_image = widgets.Output(layout=widgets.Layout(width='50%', min_height='500px', border='1px solid #ddd'))
out_report = widgets.Output(layout=widgets.Layout(width='50%', min_height='500px', border='1px solid #ddd', padding='15px', overflow='auto'))

current_img_cache = None

# ==========================================
# 2. LOGIC FUNCTIONS
# ==========================================

def get_filtered_data():
    min_age, max_age = age_slider.value
    sex = sex_dropdown.value
    mask = (df_meta['Patient Age'] >= min_age) & (df_meta['Patient Age'] <= max_age)
    if sex != 'All':
        mask = mask & (df_meta['Patient Sex'] == sex)

    # Sort: Put Positives (Actual=1) at the top, then high probabilities
    return df_meta[mask].copy().sort_values(['Actual', 'Probability'], ascending=[False, False])

def update_table(change=None):
    df_subset = get_filtered_data()

    with out_table:
        clear_output(wait=True)
        if len(df_subset) == 0:
            print("No data found.")
            return

        html_rows = ""
        # Limit to 200 rows for the selector table
        for _, row in df_subset.head(200).iterrows():

            # Styling based on Ground Truth
            is_pos = (row['Actual'] == 1)
            bg_color = "#fff0f0" if is_pos else "#f0fff0"
            status_color = "#cc0000" if is_pos else "green"
            status_text = "POSITIVE" if is_pos else "Negative"
            fw = "bold" if is_pos else "normal"

            # Styling for Score
            score = row['Probability']
            score_color = "black"
            if score > 0.7: score_color = "red"
            elif score < 0.3: score_color = "blue"

            js_click = f"google.colab.kernel.invokeFunction('select_case_gemma', [{row['ID']}], {{}})"

            html_rows += f'''
            <tr style="background-color:{bg_color}; cursor:pointer; border-bottom:1px solid #e0e0e0;" onclick="{js_click}" onmouseover="this.style.backgroundColor='#ffffcc'" onmouseout="this.style.backgroundColor='{bg_color}'">
                <td style="padding:8px;">{row['ID']}</td>
                <td style="padding:8px; font-weight:bold; color:{score_color};">{score:.3f}</td>
                <td style="padding:8px; color:{status_color}; font-weight:{fw};">{status_text}</td>
                <td style="padding:8px;">{row['Patient Age']}</td>
                <td style="padding:8px;">{row['Patient Sex']}</td>
                <td style="padding:8px; color:#999;">Click to Load ➡</td>
            </tr>'''

        table_html = f"""
        <style> th {{ position: sticky; top: 0; background: white; z-index: 10; border-bottom: 2px solid #333; }} </style>
        <table style="width:100%; border-collapse:collapse; font-family: 'Segoe UI', sans-serif; font-size:13px; text-align:center;">
            <thead>
                <tr style="background:#f8f9fa;">
                    <th style="padding:10px;">ID</th>
                    <th style="padding:10px;">Model Score</th>
                    <th style="padding:10px;">Ground Truth</th>
                    <th style="padding:10px;">Age</th>
                    <th style="padding:10px;">Sex</th>
                    <th style="padding:10px;">Action</th>
                </tr>
            </thead>
            <tbody>{html_rows}</tbody>
        </table>
        """
        display(HTML(table_html))

def load_case_image(id_val):
    global current_img_cache
    idx = int(id_val)
    id_input.value = idx
    row_subset = df_meta[df_meta['ID'] == idx]

    with out_report: clear_output(); print("Waiting for analysis...")

    with out_image:
        clear_output(wait=True)
        if len(row_subset) == 0: print("ID not found."); return
        row = row_subset.iloc[0]
        try:
            img = get_image_from_gcs(row['gcs_path'], project_id=None)
            if img:
                current_img_cache = img
                plt.figure(figsize=(7, 7))
                plt.imshow(img, cmap='gray')
                plt.axis('off')

                status_text = "POS" if row['Actual'] == 1 else "NEG"
                color = "#d63031" if row['Actual'] == 1 else "#00b894"
                title_str = f"ID: {idx}  |  Score: {row['Probability']:.3f}  |  Truth: {status_text}"

                plt.title(title_str, color=color, fontsize=15, fontweight='bold', pad=10)
                plt.show()
            else:
                print("Image load returned None.")
        except Exception as e:
            print(f"Error loading image: {e}")

def run_inference(b):
    if current_img_cache is None:
        with out_report: print("Please select an image from the table first.")
        return

    with out_report:
        clear_output()
        display(HTML("<b>MedGemma is analyzing...</b> <span style='color:gray'>(approx 10-20s)</span>"))
        display(widgets.IntProgress(value=0, min=0, max=10, layout=widgets.Layout(width='200px')))

        try:
            messages = [
                {"role": "user", "content": [
                    {"type": "text", "text": txt_prompt.value},
                    {"type": "image", "image": current_img_cache}
                ]}
            ]
            output = pipe(text=messages, max_new_tokens=500)
            response = output[0]["generated_text"][-1]["content"]

            clear_output()
            display(Markdown("### 🩺 MedGemma Report"))
            display(Markdown("---"))
            display(Markdown(response))

        except Exception as e:
            clear_output()
            print(f"Error: {e}")

output.register_callback('select_case_gemma', load_case_image)

# ==========================================
# 3. LAYOUT ASSEMBLY
# ==========================================

age_slider.observe(update_table, names='value')
sex_dropdown.observe(update_table, names='value')
id_input.observe(lambda c: load_case_image(c['new']), names='value')
btn_generate.on_click(run_inference)

controls_box = widgets.HBox([
    widgets.HTML("<b>Filter Patients:</b>"),
    age_slider,
    sex_dropdown
], layout=widgets.Layout(align_items='center', margin='10px 0', padding='10px', border='1px solid #eee'))

prompt_area = widgets.VBox([
    widgets.HTML("<b>Radiologist Prompt:</b>"),
    txt_prompt,
    btn_generate
], layout=widgets.Layout(margin='0 0 20px 0'))

workspace = widgets.HBox([
    out_image,
    widgets.VBox([
        prompt_area,
        widgets.HTML("<b>AI Findings:</b>"),
        out_report
    ], layout=widgets.Layout(width='50%', padding='0 0 0 20px'))
])

ui_gemma = widgets.VBox([
    widgets.HBox([header_html], layout=widgets.Layout(border_bottom='2px solid #e67e22', padding='15px 0')),
    controls_box,
    widgets.HTML("<b>Select a Case (Sorted by Condition):</b>"),
    out_table,
    widgets.HTML("<hr>"),
    widgets.HBox([widgets.Label("Selected ID:"), id_input]),
    workspace
])

update_table()
display(ui_gemma)